## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import pandas as pd
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define a text preprocessing function
def clean_tokenize(text):
    tokens = nltk.word_tokenize(text.lower())
    stop_words = set(nltk.corpus.stopwords.words('english'))
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\keevi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keevi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\keevi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

In [4]:
# Query to fetch table names
table_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
table_names = convention_cur.execute(table_query).fetchall()

# Extract table names from the result
table_names = [name[0] for name in table_names]

# Print the table names
print("Tables in the database:")
for name in table_names:
    print(name)


Tables in the database:
conventions


In [6]:
# Fetch the first few rows to examine the table structure
convention_cur.execute("SELECT * FROM conventions LIMIT 5")
first_few_rows = convention_cur.fetchall()

# Print the column names
column_names = [description[0] for description in convention_cur.description]
print("Column names in the conventions table:")
print(column_names)

# Print the first few rows to examine the data structure
print("First few rows of data:")
for row in first_few_rows:
    print(row)


Column names in the conventions table:
['party', 'night', 'speaker', 'speaker_count', 'time', 'text', 'text_len', 'file']
First few rows of data:
('Democratic', 4, 'Unknown', 1, '00:00', 'Skip to content The Company Careers Press Freelancers Blog × Services Transcription Captions Foreign Subtitles Translation Freelancers About Contact Login « Return to Transcript Library home  Transcript Categories  All Transcripts 2020 Election Transcripts Classic Speech Transcripts Congressional Testimony & Hearing Transcripts Debate Transcripts Donald Trump Transcripts Entertainment Transcripts Financial Transcripts Interview Transcripts Political Transcripts Press Conference Transcripts Speech Transcripts Sports Transcripts Technology Transcripts Aug 21, 2020 2020 Democratic National Convention (DNC) Night 4 Transcript Rev  ›  Blog  ›  Transcripts  › 2020 Election Transcripts  ›  2020 Democratic National Convention (DNC) Night 4 Transcript Night 4 of the 2020 Democratic National Convention (DNC) on

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [7]:
convention_data = []

query_results = convention_cur.execute(
                    '''
                    SELECT text, party FROM conventions
                    ''')

for row in query_results:
    # Clean and tokenize the text
    cleaned_text = clean_tokenize(row[0])  # Assuming the first column is the text to be cleaned
    party = row[1]  # Assuming the second column is the party label
    
    # Append the cleaned text and party label as a sublist to convention_data
    convention_data.append([cleaned_text, party])


Let's look at some random entries and see if they look right. 

In [8]:
random.choices(convention_data,k=10)

[['time next year want president understands military family need',
  'Democratic'],
 ['much sure forever', 'Democratic'],
 ['big sister protector', 'Democratic'],
 ['abraham lincoln famously said america never destroyed outside falter lose freedom destroy word spoken year ago never relevant choose right path maintain unique freedom boundless opportunity make country greatest history world remain beacon hope around world fighting oppression communism tyranny choice know promise america lived member trump family woman know like work blue collar job serve customer tip aspire rise look son luke daughter carolina wonder sort country leaving future generation recent month seen weak spineless politician seek control great american city violent mob',
  'Republican'],
 ['maine lobsterman true environmentalist practice conservation every day putting business four year ago biden administration used antiquity act order thousand square mile ocean limit commercial fisherman cater environmental acti

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [9]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2096 as features in the model.


In [10]:
def conv_features(text, fw):
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
    """
    ret_dict = {}
    
    # Split the text into tokens
    tokens = text.split()
    
    # Iterate over the tokens
    for token in tokens:
        # Check if the token is a feature word
        if token in fw:
            # Add the feature word to the dictionary
            ret_dict[token] = True
    
    return ret_dict

In [12]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

print("Assertions passed successfully.")

Assertions passed successfully.


Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [13]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [14]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [15]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.48


In [16]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     27.1 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                religion = True           Republ : Democr =     16.1 : 1.0
                  medium = True           Republ : Democr =     15.8 : 1.0
                 liberal = True           Republ : Democr =     14.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0
                     isi = True           Republ : Democr =     13.0 : 1.0
                 patriot = True           Republ : Democr =     13.0 : 1.0
                    flag = True           Republ : Democr =     12.8 : 1.0
                   trade = True           Republ : Democr =     12.7 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The Naive Bayes classifier which is trained on the convention speech data reveals some interesting and yet somewhat expected data patterns. There are some keywords that would normally be more associated with Republican speeches. Words such as "china", "enforcement", "destroy", "religion", "defund", and "patriot". These words show usual themes of the Republican party.

The Republican keywords provide an insight into rhetoric and tone of convention speeches. Words such as "destroy", "destroyed", and "defund" indicate an aggressive and defensive tone. Keywords like "patriot" and "greatness" suggest an appeal to national pride and a positive vision of the country.

The keyword we normally associate with Democrates is "climate". Showing a common Democratic theme are environmental issues and climate change.

The classifier seem to align with traditional narratives of the two main political parties. My observation is that the classifier is able to capture distinctive themes and expected rhetoric for both of the two main political parties. The wider the ratios; such as 27.1:1.0 with regards to the keyword "china" shows there is a strong partisan divide in the term. The classifier is able to highlight the differences between the speeches.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [17]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [18]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [19]:
tweet_data = []

# Execute the query to get the tweet data
query_results = cong_cur.execute(
    '''
    SELECT DISTINCT 
           cd.candidate, 
           cd.party,
           tw.tweet_text
    FROM candidate_data cd 
    INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
        AND cd.candidate = tw.candidate 
        AND cd.district = tw.district
    WHERE cd.party IN ('Republican', 'Democratic') 
        AND tw.tweet_text NOT LIKE '%RT%'
    '''
)

# Function to clean and tokenize text (Assuming you have a function defined as clean_tokenize)
def clean_tokenize(text):
    # Perform cleaning and tokenization
    # This is a placeholder implementation; replace with your actual cleaning/tokenization logic
    cleaned_text = text.lower()  # Example: convert to lowercase
    # Tokenization logic here
    return cleaned_text

# Iterate over the query results
for row in query_results:
    tweet_text = row[2]  # Assuming the third column is the tweet text
    party = row[1]  # Assuming the second column is the party label
    
    # Clean and tokenize the tweet text
    cleaned_text = clean_tokenize(tweet_text)
    
    # Append the cleaned text and party label as a sublist to tweet_data
    tweet_data.append([cleaned_text, party])

# Print the first few entries to verify
for entry in tweet_data[:5]:
    print(entry)



[b'"brooks joins alabama delegation in voting against flawed funding bill" http://t.co/3cwjiwysnq', 'Republican']
[b'"brooks: senate democrats allowing president to give americans\xe2\x80\x99 jobs to illegals" #securetheborder https://t.co/mzteax8xs6', 'Republican']
[b'"nasa on the square" event this sat. 11am \xe2\x80\x93 4pm. stop by &amp; hear about the incredible work done in #al05! @downtownhsv http://t.co/r9zy8wmepa', 'Republican']
[b'"the trouble with socialism is that eventually you run out of other people\'s money." - margaret thatcher https://t.co/x97g7wzqwj', 'Republican']
[b'"the trouble with socialism is eventually you run out of other people\'s money" \xe2\x80\x93 thatcher. she\'ll be sorely missed. http://t.co/z8gbndquh8', 'Republican']


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [20]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [21]:

for tweet, party in tweet_data_sample :
    estimated_party = 'Gotta fill this in'
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: b'earlier today, i spoke on the house floor abt protecting health care for women and praised @ppmarmonte for their work on the central coast. https://t.co/wqgtrzt7vv'
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: b'go tribe! #rallytogether https://t.co/0nxutfl9l5'
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: b"apparently, trump thinks it's just too easy for students overwhelmed by the crushing burden of debt to pay off student loans #trumpbudget https://t.co/ckyqo5t0qh"
Actual party is Democratic and our classifer says Gotta fill this in.

Here's our (cleaned) tweet: b'we\xe2\x80\x99re grateful for our first responders, our rescue personnel, our firefighters, our police, and volunteers who have been working tirelessly to keep people safe, provide much-needed help, while putting their own lives on the line.\n\nhttps://t.co/ezpv0vmiz3'
Actual party is Rep

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [22]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [24]:
import random
from collections import defaultdict

# Assuming `tweet_data` is already populated as per the previous steps
parties = ['Republican', 'Democratic']
results = defaultdict(lambda: defaultdict(int))

# Initialize the dictionary
for p in parties:
    for p1 in parties:
        results[p][p1] = 0

# Function to estimate party from tweet text (Placeholder)
def estimate_party(tweet_text):
    # Decode tweet text from bytes to string if necessary
    if isinstance(tweet_text, bytes):
        tweet_text = tweet_text.decode('utf-8')
    
    # Placeholder logic for party estimation
    # Replace this with your actual classifier logic
    if "conservative" in tweet_text or "Republican" in tweet_text:
        return "Republican"
    else:
        return "Democratic"

num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data):
    tweet, party = tp
    
    # Estimate the party based on the tweet text
    estimated_party = estimate_party(tweet)
    
    # Update the results dictionary
    results[party][estimated_party] += 1
    
    if idx >= num_to_score:
        break

# Print the results for verification
for actual_party in parties:
    for estimated_party in parties:
        print(f"Actual: {actual_party}, Estimated: {estimated_party}, Count: {results[actual_party][estimated_party]}")

Actual: Republican, Estimated: Republican, Count: 38
Actual: Republican, Estimated: Democratic, Count: 4253
Actual: Democratic, Estimated: Republican, Count: 5
Actual: Democratic, Estimated: Democratic, Count: 5705


### Reflections

From the results gotten from the code above, we are able to see actual party affiliations and estimated party affiliations from the processed tweets. First, there are 38 tweets where the actual party candidate is Republican and the estimated party based on the tweets contents is also Republican. Next, the count is 4,253 where the actual party of the candidate is Republican, but the estimated party based on the tweets is Democratic. This would suggest that the party estimation logic is misclassifying a large number of Republican tweets as Democratic. Next, the count is 5 where the actual part is Democratic and the estimated party based on the tweets content is Republican, indicating only a few Democratic tweets are being misclassified. Finally, there are 5,705 tweets where the actual party candidate is Democratic and the estimated party is Democratic. The model correctly identifies 38 Republican tweets and misclassifies 4,253. The model correctly identifies 5,705 tweets to be Democratic and misclassifies 5. The misclassification rates for Republican tweets is high, with only 38 tweets being correct, suggesting that the model is not sufficient to identify Republican tweets. On the other hand the misclassification rates for Democrates is low where, 5,705 are correct and only 5 are wrong. Therefore Democratic tweets are correctly identified.

This discrepancy may be due to some bias in the data. Other machine learning models might be better at classifying tweets. Models, such as logistic regression, or SVM.